In [4]:
import numpy as np
import pandas as pd

In [5]:
temp_df = pd.read_csv("../data/IMDB Dataset.csv")

In [6]:
temp_df.shape

(50000, 2)

In [7]:
df = temp_df[:10000]

In [8]:
df.shape

(10000, 2)

In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df.drop_duplicates(inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2784\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [11]:
df.duplicated().sum()

0

In [12]:
import re
def remove_tags(text):
    cleaned_text = re.sub(re.compile('<.*?>'),"",text)
    return cleaned_text

In [13]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2784\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [14]:
df['review']

0       One of the other reviewers has mentioned that ...
1       A wonderful little production. The filming tec...
2       I thought this was a wonderful way to spend ti...
3       Basically there's a family where a little boy ...
4       Petter Mattei's "Love in the Time of Money" is...
                              ...                        
9995    Fun, entertaining movie about WWII German spy ...
9996    Give me a break. How can anyone say that this ...
9997    This movie is a bad movie. But after watching ...
9998    This is a movie that was probably made to ente...
9999    Smashing film about film-making. Shows the int...
Name: review, Length: 9983, dtype: object

In [15]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\Admin\AppData\Local\Temp\ipykernel_2784\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [16]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

st_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [17]:
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in st_words]).apply(lambda x: " ".join(x))

C:\Users\Admin\AppData\Local\Temp\ipykernel_2784\647783855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in st_words]).apply(lambda x: " ".join(x))


In [18]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [19]:
import gensim

In [20]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [21]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [22]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [23]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2
)

In [24]:
model.build_vocab(story)

In [25]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875767, 6212140)

In [26]:
len(model.wv.index_to_key)

31845

In [27]:
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [28]:
document_vector(df['review'].values[0])

array([-0.15486121,  0.4360582 ,  0.15324074,  0.24008678, -0.18509993,
       -0.6428462 ,  0.18475929,  0.9207185 , -0.36446738, -0.29321986,
       -0.3180964 , -0.46224123,  0.09622049,  0.12381569,  0.17559533,
       -0.10870445, -0.01810482, -0.33805084, -0.06161341, -0.6171721 ,
        0.04305154,  0.24443652,  0.11718043, -0.29601446, -0.3407403 ,
       -0.02520885, -0.3608657 , -0.01235519, -0.347503  ,  0.04097877,
        0.33200693,  0.01351151,  0.24537665, -0.2894824 , -0.1104169 ,
        0.3995561 ,  0.10695299, -0.35364005, -0.2640086 , -0.7788672 ,
        0.12603813, -0.23846991,  0.06181221, -0.09948052,  0.50947434,
       -0.14909221, -0.27635375, -0.06411703,  0.14677194,  0.35153517,
        0.05540048, -0.33638617, -0.4267444 , -0.11699168, -0.08882901,
        0.19241813,  0.23395982,  0.00288069, -0.30508462,  0.12754703,
        0.0764136 ,  0.13060413,  0.02438384, -0.08281149, -0.45717797,
        0.28694475,  0.05657283,  0.05516612, -0.37458152,  0.27

In [29]:
from tqdm import tqdm

In [30]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [19:50<00:00,  8.38it/s]


In [31]:
X = np.array(X)
X[0]

array([-0.15486121,  0.4360582 ,  0.15324074,  0.24008678, -0.18509993,
       -0.6428462 ,  0.18475929,  0.9207185 , -0.36446738, -0.29321986,
       -0.3180964 , -0.46224123,  0.09622049,  0.12381569,  0.17559533,
       -0.10870445, -0.01810482, -0.33805084, -0.06161341, -0.6171721 ,
        0.04305154,  0.24443652,  0.11718043, -0.29601446, -0.3407403 ,
       -0.02520885, -0.3608657 , -0.01235519, -0.347503  ,  0.04097877,
        0.33200693,  0.01351151,  0.24537665, -0.2894824 , -0.1104169 ,
        0.3995561 ,  0.10695299, -0.35364005, -0.2640086 , -0.7788672 ,
        0.12603813, -0.23846991,  0.06181221, -0.09948052,  0.50947434,
       -0.14909221, -0.27635375, -0.06411703,  0.14677194,  0.35153517,
        0.05540048, -0.33638617, -0.4267444 , -0.11699168, -0.08882901,
        0.19241813,  0.23395982,  0.00288069, -0.30508462,  0.12754703,
        0.0764136 ,  0.13060413,  0.02438384, -0.08281149, -0.45717797,
        0.28694475,  0.05657283,  0.05516612, -0.37458152,  0.27

In [32]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()


In [43]:
Y = encoder.fit_transform(df['sentiment'])

In [44]:
Y


array([1, 1, 1, ..., 0, 0, 1])

In [45]:
print(X.shape)
print(Y.shape)

(9983, 100)
(9983,)


In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [46]:
rf =RandomForestClassifier()
rf.fit(x_train,y_train)


RandomForestClassifier()

In [49]:
y_pred = rf.predict(x_test)
accuracy_score(y_test,y_pred)

0.757135703555333